In [1]:
import nltk
import os
from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.tag import StanfordPOSTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
java_path = "C:/Program Files/Java/jdk1.8.0_144/bin/java.exe"
os.environ['JAVAHOME'] = java_path
model='stanford-postagger-2018-02-27/models/english-left3words-distsim.tagger'
jar='stanford-postagger-2018-02-27/stanford-postagger-3.9.1.jar'
st=StanfordPOSTagger(model,jar,encoding='utf-8')
lemmatizer=WordNetLemmatizer()
punctuations = '''!()-[]{};—:'"\,<>./’?@#$%^&*_~“”'''

c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\nltk\tag\stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [2]:
stopwords=['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves','he','him','his','himself'
,'she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom',
 'this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a',
 'an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','between','into','through','during',
 'before','after','to','from','again','then','once','here','there','when','where','why','how','all','any','both','each',
 'than','should','d','ll','m','o','re','ve','y','ma']
stopwords=stopwords+list(punctuations)

In [3]:
neg= {"aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never","no", "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom","minimise", "despite","unfulfilled","undue","anti","against","without"}

In [4]:
boost={"absolutely":0.5, "amazingly":0.125, "awfully":0.25, "completely":0.25, "considerably":0.125,
 "decidedly":0.25 , "deeply":0.25 , "enormously":0.25 ,
 "entirely":0.5 , "especially":0.25 , "exceptionally":0.25, "extremely":0.625 ,"fabulously":0.25   , "fully":0.375  ,
 "greatly":0.125 , "highly":0.5   ,"huge":0.25, "hugely":0.25   , "incredibly":0.25   ,
 "intensely":0.25   , "majorly":0.625  ,"particularly":0.125 ,"really":0.375 ,"substantially":0.125   ,
 "thoroughly":0.625   , "totally":0.5  ,"unbelievably":0.25   , "utterly":0.5   ,
 "very":0.25  ,"tremendously":-0.25 ,  "barely":-0.375   ,"badly":-0.125 ,"hardly":-0.25,"unusually":-0.5   , 
 "less":-0.5   , "little":-0.375   , "marginally":-0.125   , "partly":-0.125   ,
 "scarcely":-0.25   , "slightly":-0.25   , "somewhat":-0.125, "shoddy":-0.625, "poorly":-0.75}
pos_boost=["absolutely", "amazingly", "awfully", "completely", "considerably",
 "decidedly","deeply","enormously","entirely","especially","exceptionally","extremely","fabulously","fully",
 "greatly","highly","huge","hugely", "incredibly","intensely","majorly","particularly","really","substantially","thoroughly","totally","unbelievably", "utterly","very"]
neg_boost=["tremendously", "barely","badly","hardly","unusually", "less", "little", "marginally", "partly",
 "scarcely", "slightly","somewhat", "shoddy", "poorly"]

In [6]:
def convert_tag(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [8]:
punctuations = '''!@'#-+=_'''
def clean_tweet(t):
    for c in punctuations:
             t= t.replace(c," ")
    return t


In [9]:
def positive_booster(booster,word,pos2):
    sec_score=get_sentiment(word,pos2)
    if sec_score>0.0:
        return(boost[booster]+sec_score)
    if sec_score<0.0:
        return(sec_score-boost[booster])
    else:
        return(boost[booster])

In [10]:
def negative_booster(booster,word,pos2):
    sec_score=get_sentiment(word,pos2)
    if sec_score>0.0:
        return(sec_score+boost[booster])
    if sec_score<0.0:
        return(sec_score+boost[booster])
    else:
        return(boost[booster])

In [11]:
def get_sentiment(word,wn_tag):
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        synsets = wn.synsets(lemma, pos=wn_tag)
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())
    #print(word,synset,swn_synset.pos_score(),swn_synset.neg_score())
    return(swn_synset.pos_score() - swn_synset.neg_score())
    

In [12]:
def normalize(score):
    if score>=0.0:
        return((score-(0.0))/2.75)
    else:
        return(-(-score-(0.125))/1.25)

In [13]:
tweet1='''The marginalization of the Tax Cheats which was expected post #GST will now happen in right earnest as #Ewaybill rolls out. Real benefits to most organized sector companies will be seen in FY2019'''
tweet2='''From Failed #Demonetisation to Shoddy Implementation of #GST to poorly performing #Economy to unfulfilled Promise of reduction in fuel prices to not keeping the promises of providing 2 crores jobs, BJP has wrecked a nation & has broken people’s faith '''
tweet3='''There is nothing wrong with #GST implementation but the fact is we are all losers. Injected to us by @INCIndia  Even God cannot change'''
tweet4='''So, Indian businesses are mad against GST cause now they have to pay tax and earlier they weren't?'''
tweet5='''#GST in india is total failure,it like nightmare for tax payers Indians..in my view it could be rolled out after #2019LSPolls after corrections ~ Dr Subramanian @Swamy39 Speaking at 14th Annual 🇮🇳India Business Conference on ‘Indian Growth @Columbia_Biz '''
tweet6='''U know u r abt to witness #GST_Mess when a common citizen is forced to pay 28% #GST on BANANAS. '''

In [14]:
A=word_tokenize(clean_tweet(tweet6.lower()))
bigrams=nltk.ngrams(A,2)

In [13]:
i=0
not_found=set()
score=0.0
for b in bigrams:
    i+=1
    #b_tag=pos_tag(b)
    b_tag=st.tag(b)
    #print(b_tag)
    second_gram=b_tag[1]
    first_gram=b_tag[0]
    if second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN') or second_gram[1].startswith('VBD'):
    #if second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VB'):
        pos2=convert_tag(second_gram[1])
        pos1=convert_tag(first_gram[1])
        
        if first_gram[0] in boost:
            if first_gram[0] in pos_boost:
                score=positive_booster(first_gram[0],second_gram[0],pos2)
                print('Pos-Boost:',b_tag,score)
            if first_gram[0] in neg_boost:
                score=negative_booster(first_gram[0],second_gram[0],pos2)
                print('Neg-Boost:',b_tag,score)
            
   
        if first_gram[0] in neg:
                sec_score=get_sentiment(second_gram[0],pos2)
                print(second_gram[0],sec_score)
                if sec_score ==0.0 or sec_score==None:
                    score+=(get_sentiment(first_gram[0],pos1)+sec_score)
                else:
                    score+=-(sec_score)
                print('Neg:',b_tag,score)
            
        elif first_gram[0] not in neg and first_gram[0] not in boost:
            try:
                first_score=get_sentiment(first_gram[0],pos1)
                score+=first_score+get_sentiment(second_gram[0],pos2)
                print('Normal:',b_tag,score)
            except:
                print("Not Found:",second_gram[0])
                not_found.add(second_gram[0])
    
print(score)

           

Not Found: mess
a Synset('angstrom.n.01') 0.0 0.0
common Synset('common.a.01') 0.0 0.0
Normal: [('a', 'DT'), ('common', 'JJ')] 0.0
is Synset('be.v.01') 0.25 0.125
forced Synset('coerce.v.01') 0.375 0.125
Normal: [('is', 'VBZ'), ('forced', 'VBN')] 0.375
0.375


In [14]:
def check(bigrams):
    score=0.0
    for b in bigrams:
        b_tag=st.tag(b)
        #b_tag=pos_tag(b)
        #print(b_tag)
        second_gram=b_tag[1]
        first_gram=b_tag[0]
        if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN') or second_gram[1].startswith('VBD'):
            pos2=convert_tag(second_gram[1])
            pos1=convert_tag(first_gram[1])
        
            if first_gram[0] in boost:
                if first_gram[0] in pos_boost:
                    score=positive_booster(first_gram[0],second_gram[0],pos2)
                    print('Pos-Boost:',b_tag,score)
                if first_gram[0] in neg_boost:
                    score=negative_booster(first_gram[0],second_gram[0],pos2)
                    print('Neg-Boost:',b_tag,score)
            
   
            if first_gram[0] in neg:
                x=0.0
                sec_score=get_sentiment(second_gram[0],pos2)
                print(second_gram[0],sec_score)
                if sec_score ==0.0 or sec_score==None:
                    try:
                        x=get_sentiment(first_gram[0],pos1)
                        score+=(x+sec_score)
                    except:
                        print(x)
                else:
                    score+=-(sec_score)
                #print('Neg:',b_tag,score)
            
            elif first_gram[0] not in neg and first_gram[0] not in boost:
                try:
                    first_score=get_sentiment(first_gram[0],pos1)
                    score+=first_score+get_sentiment(second_gram[0],pos2)
                    #print('Normal:',b_tag,score)
                except:
                    print("Not Found:",second_gram[0])
                    #not_found.add(second_gram[0])
    return (normalize(score))
    
    

In [15]:
path="D:/LEX/ProjectS-master/tweet/"
tweet_list=os.listdir(path)
#fw=open("tweets.txt","r",encoding="utf-8")
rw=open("sentiwordnet_normalized_day_tweet_result.txt","a",encoding='utf-8')
#tw=fw.read().split('\n')
for i in tweet_list:
    score=0.0
    file=open(path+i,"r",encoding='utf-8')
    lines=file.readlines()
    for l in lines:
        tok_tweet=word_tokenize(clean_tweet(l.lower()))
        bigrams=nltk.ngrams(tok_tweet,2)
        score+=check(bigrams)
    rw.write(str(i)+":"+str(score/len(lines))+"\n")
    print(i)
rw.close()
'''
fw=open("tweets.txt","r",encoding="utf-8")
rw=open("norm_stan_tweet_result.txt","a",encoding='utf-8')
tw=fw.read().split('\n')
i=0
for t in tw:
    i+=1
    score=0.0
    tok_tweet=word_tokenize(clean_tweet(t.lower()))
    bigrams=nltk.ngrams(tok_tweet,2)
    score=check(bigrams)
    rw.write(str(t)+"  :"+str(score)+"\n")
    print(i)
fw.close()
rw.close()
'''

Not Found: expected
Not Found: pain
Not Found: indian
Not Found: usual
Not Found: promised
Not Found: handling
Not Found: economic
doubt 0.125
Not Found: more
01-07-2017.txt
Not Found: mess
Not Found: depressing
Not Found: most
Not Found: indian
Not Found: perfect
Not Found: greater
Not Found: permanent
Not Found: boost
Not Found: positive
Not Found: expected
Not Found: pain
Not Found: indian
02-07-2017.txt
Not Found: faith
Not Found: nuclear
Not Found: proud
Not Found: indian
Not Found: supporting
Not Found: little
Neg-Boost: [('little', 'JJ'), ('demonetisation', 'NN')] -0.875
Not Found: did
Not Found: greatness
Not Found: aborted
more 0.0
Not Found: biggest
03-07-2017.txt
Not Found: strange
Not Found: multiple
Not Found: .indian
Not Found: central
Not Found: similar
Not Found: many
Not Found: brilliance
Not Found: many
Not Found: struggling
Not Found: new
Not Found: seperate
04-07-2017.txt
Not Found: issues
Pos-Boost: [('huge', 'JJ'), ('discomfort', 'NN')] -0.5
Not Found: unable
Not 

NameError: name 'fw' is not defined

In [20]:
rw.close()

In [ ]:
fo=open("Result/Tweets/SentiWordnet/Sentiwordnet_stan_tweet_result.txt","r",encoding='utf-8')
lines=fo.readlines()
s_list=[]
for l in lines:
    #print(l)
    s_list.append(float(l.split(':')[1]))
fo.close()

In [24]:
pos_list,neg_list=[],[]
for s in s_list:
    if s >=0.0:
        pos_list.append(s)
    if s<0.0:
        neg_list.append(-s)

In [31]:
max(neg_list)

1.375

In [32]:
1.375-0.125

1.25